# **Masfudin Dwi Cahyo**

### **masfudindc@gmail.com**



---



# **Submission Dicoding**

## **Proyek Akhir : Klasifikasi Gambar**


---



# **Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load data bertipe zip dengan library zipfile
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/Dicoding/rockpaperscissors.zip', 'r') as zip:
  zip.extractall('/content/drive/MyDrive/Dicoding/rockpaperscissors')

# **Import Library**